In [1]:
import numpy as np
import pandas as pd 
import jieba 
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from collections import Counter
from math import isnan
from sklearn.feature_extraction.text import TfidfTransformer
txt = pd.read_excel('guo(new_3w).xlsx').drop('空白字段',axis = 1)
for i in range(len(txt)):
    length = len(txt.loc[i]['商品描述'].split())
    if length >15:
        txt.drop(index = i,inplace=True)
frame = pd.DataFrame(columns=['商品描述','标签'])
dic = Counter(txt['标签'])
for i in dic.keys():
    data = txt.loc[txt['标签'] == i]
    if dic[i] > 200:
        frame = pd.concat([frame,data[:200]],axis = 0)
        continue
    elif dic[i]>=10:
        frame = pd.concat([frame,data],axis = 0)
frame = frame.reset_index(drop= True)
c_num = len(Counter(frame['标签']).keys())
print('数据量：%d'%len(frame),'类目数：%d'%len(Counter(frame['标签'])))

c:\python3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


数据量：21193 类目数：510


In [2]:
word_txt = []
for label in Counter(frame['标签']).keys():
    same = frame.loc[frame['标签'] == label]
    word = []
    for i in same['商品描述']:
        word += i.split()
    word_txt.append(word)
from gensim.models import word2vec
model = word2vec.Word2Vec(word_txt,size = 100,window = 900 ,min_count = 3)
word2vec_matrix = []
for line in range(frame.shape[0]):
    vector = []
    num = 0
    words = frame['商品描述'][line].split()
    for word in words:
        try:
            temp = model[word]
        except:
            continue 
        else:
            vector += list(temp)
            num += 1
    vector = vector + [0]*100*(15 - num)
    word2vec_matrix.append(vector)
from sklearn import preprocessing 
le = preprocessing.LabelEncoder()
y_label = le.fit_transform(frame['标签'])

c:\python3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
c:\python3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [3]:
import tensorflow as tf
from tensorflow.contrib import rnn
import random

input_size = 100
timestep_size = 15 
hidden_size = 256
layer_num = 2
class_num = c_num
learning_rate = 0.001
with tf.name_scope('inputs'):
    keep_prob = tf.placeholder(tf.float32)
    batch_size = tf.placeholder(tf.int32, [], name='batch_size_input')
    _X = tf.placeholder(tf.float32,[None,1500])
    y = tf.placeholder(tf.int64,[None])
    x = tf.reshape(_X,[-1,timestep_size,input_size])

with tf.name_scope("fw"):
    stacked_rnn_fw = []
    for _ in range(layer_num):
        fw_cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size, forget_bias=1.0, state_is_tuple=True)
        stacked_rnn_fw.append(fw_cell)
    lstm_fw_cell_m = tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.MultiRNNCell(stacked_rnn_fw), output_keep_prob= keep_prob)
    #lstm_fw_cell_m = tf.nn.rnn_cell.MultiRNNCell(cells=stacked_rnn_fw, state_is_tuple=True)

with tf.name_scope("bw"):
    stacked_rnn_bw = []
    for _ in range(layer_num):
        bw_cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size, forget_bias=1.0, state_is_tuple=True)
        stacked_rnn_bw.append(bw_cell)
    lstm_bw_cell_m = tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.MultiRNNCell(stacked_rnn_bw), output_keep_prob= keep_prob)
    #lstm_bw_cell_m = tf.nn.rnn_cell.MultiRNNCell(cells=stacked_rnn_bw, state_is_tuple=True)
    
with tf.name_scope("output"):
    x = tf.unstack(x, 15, axis=1)
    outputs, _, _ = rnn.static_bidirectional_rnn(lstm_fw_cell_m, lstm_bw_cell_m, x, dtype=tf.float32)
with tf.name_scope("result"):
    w = tf.Variable(tf.random_uniform([2 * hidden_size, class_num], -1.0, 1.0), name='W')
    b = tf.Variable(tf.constant(0.1,shape=[class_num]), dtype=tf.float32)
    logits = tf.nn.xw_plus_b(outputs[-1], w, b)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits+1e-10,labels = y)
    original_cost_function = tf.reduce_mean(loss)
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(original_cost_function)
with tf.name_scope("accuracy"):
    prediction = tf.argmax(logits,1)
    correct_prediction = tf.equal(prediction,y)
    correct_num=tf.reduce_sum(tf.cast(correct_prediction,tf.float32))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32),name="accuracy")

In [4]:
X_train,X_test,Y_train,Y_test = train_test_split(word2vec_matrix,y_label,test_size = 0.1)

In [5]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    listnum = []
    Iter = 0
    for i in range(10001):
        _batch_size = 1000
        if len(listnum)<_batch_size:
            listnum = list(range(len(X_train)))
            Iter += 1
        temp_x =[]
        temp_y = []
        for _ in range(_batch_size):
            j = random.randint(0,len(listnum)-1) #生成一个包括0，len(listnum)-1之间的随机数
            temp_x.append(X_train[listnum[j]])
            temp_y.append(Y_train[listnum[j]])
            del listnum[j]
        # temp_x 是一个1000*1200维度的向量
        if (i)%200 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={_X:X_test, y: Y_test, keep_prob: 1.0, batch_size:len(X_test)})
            print("Iter %d: step %d, training accuracy %g" % ( Iter,(i+1), train_accuracy))
        sess.run(train_op,feed_dict={_X:temp_x,y:temp_y,keep_prob: 0.5, batch_size: _batch_size})

Iter 1: step 1, training accuracy 0.000943396
Iter 11: step 201, training accuracy 0.829717
Iter 22: step 401, training accuracy 0.864151
Iter 32: step 601, training accuracy 0.873113
Iter 43: step 801, training accuracy 0.877358
Iter 53: step 1001, training accuracy 0.872642


KeyboardInterrupt: 